[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/qubvel/segmentation_models.pytorch/blob/main/examples/camvid_segmentation_multiclass.ipynb)

# Install package

In [6]:
%%capture
%pip install --upgrade segmentation-models-pytorch lightning albumentations

# Loading data
For this example we will use CamVid dataset. It is a set of:
 - train images + segmentation masks
 - validation images + segmentation masks
 - test images + segmentation masks

All images have 320 pixels height and 480 pixels width. For more inforamtion about dataset visit http://mi.eng.cam.ac.uk/research/projects/VideoRec/CamVid/.

In [7]:
%%capture
# Download Data
!git clone https://github.com/alexgkendall/SegNet-Tutorial ./data

In [8]:
import os
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset

In [20]:
from google.colab import drive
drive.mount('/content/drive')

DATA_DIR = "/content/drive/MyDrive/dataset"

x_train_dir = os.path.join(DATA_DIR, "x_train")
y_train_dir = os.path.join(DATA_DIR, "y_train")
x_val_dir = os.path.join(DATA_DIR, "x_val")
y_val_dir = os.path.join(DATA_DIR, "y_val")
x_test_dir = os.path.join(DATA_DIR, "x_test")
y_test_dir = os.path.join(DATA_DIR, "y_test")


# DATA_DIR = "./data/CamVid/"

# x_train_dir = os.path.join(DATA_DIR, "train")
# y_train_dir = os.path.join(DATA_DIR, "trainannot")

# x_valid_dir = os.path.join(DATA_DIR, "val")
# y_valid_dir = os.path.join(DATA_DIR, "valannot")

# x_test_dir = os.path.join(DATA_DIR, "test")
# y_test_dir = os.path.join(DATA_DIR, "testannot")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Dataloader
Writing helper class for data extraction, tranformation and preprocessing
https://pytorch.org/docs/stable/data

In [21]:
import os
import cv2
import numpy as np
import albumentations as A
from torch.utils.data import Dataset


# RGB 色マップ → クラスID
COLOR_MAP = {
    (0, 0, 0): 0,              # background
    (198, 67, 151): 1,         # liver
    (38, 43, 104): 2,          # portal vein
    (255, 128, 0): 3           # vascular
}

CLASSES = ['background', 'liver', 'portal_vein', 'vascular']

class MedicalDataset(Dataset):
    def __init__(self, images_dir, masks_dir, augmentation=None):
        self.image_paths = sorted([os.path.join(images_dir, x) for x in os.listdir(images_dir)])
        self.mask_paths = sorted([os.path.join(masks_dir, x) for x in os.listdir(masks_dir)])
        self.augmentation = augmentation

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, i):
        image = cv2.imread(self.image_paths[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        mask_rgb = cv2.imread(self.mask_paths[i])
        mask_rgb = cv2.cvtColor(mask_rgb, cv2.COLOR_BGR2RGB)

        # RGBマスクをクラスIDマスクに変換
        mask = np.zeros((mask_rgb.shape[0], mask_rgb.shape[1]), dtype=np.uint8)
        for rgb, class_id in COLOR_MAP.items():
            mask[np.all(mask_rgb == rgb, axis=-1)] = class_id

        if self.augmentation:
            augmented = self.augmentation(image=image, mask=mask)
            image, mask = augmented['image'], augmented['mask']

        image = image.transpose(2, 0, 1).astype('float32') / 255.0
        return image, mask




# import os
# import cv2
# import numpy as np
# import albumentations as A


# class Dataset(BaseDataset):
#     CLASSES = [
#         "sky",
#         "building",
#         "pole",
#         "road",
#         "pavement",
#         "tree",
#         "signsymbol",
#         "fence",
#         "car",
#         "pedestrian",
#         "bicyclist",
#         "unlabelled",
#     ]

#     def __init__(self, images_dir, masks_dir, classes=None, augmentation=None):
#         self.ids = os.listdir(images_dir)
#         self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
#         self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]

#         # Always map background ('unlabelled') to 0
#         self.background_class = self.CLASSES.index("unlabelled")

#         # If specific classes are provided, map them dynamically
#         if classes:
#             self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
#         else:
#             self.class_values = list(range(len(self.CLASSES)))  # Default to all classes

#         # Create a remapping dictionary: class value in dataset -> new index (0, 1, 2, ...)
#         # Background will always be 0, other classes will be remapped starting from 1.
#         self.class_map = {self.background_class: 0}
#         self.class_map.update(
#             {
#                 v: i + 1
#                 for i, v in enumerate(self.class_values)
#                 if v != self.background_class
#             }
#         )

#         self.augmentation = augmentation

#     def __getitem__(self, i):
#         # Read the image
#         image = cv2.imread(self.images_fps[i])
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

#         # Read the mask in grayscale mode
#         mask = cv2.imread(self.masks_fps[i], 0)

#         # Create a blank mask to remap the class values
#         mask_remap = np.zeros_like(mask)

#         # Remap the mask according to the dynamically created class map
#         for class_value, new_value in self.class_map.items():
#             mask_remap[mask == class_value] = new_value

#         if self.augmentation:
#             sample = self.augmentation(image=image, mask=mask_remap)
#             image, mask_remap = sample["image"], sample["mask"]
#         image = image.transpose(2, 0, 1)
#         return image, mask_remap

#     def __len__(self):
#         return len(self.ids)

In [24]:
import matplotlib.pyplot as plt
import numpy as np

# クラスID → 色（RGB）の対応（背景、肝臓、門脈、血管）
COLOR_MAP = {
    0: (0, 0, 0),               # background
    1: (198, 67, 151),          # liver
    2: (38, 43, 104),           # portal vein
    3: (255, 128, 0),           # vascular
}

def decode_mask(mask):
    """
    クラスIDマスク (HxW) → RGBマスク (HxWx3) に変換
    """
    h, w = mask.shape
    color_mask = np.zeros((h, w, 3), dtype=np.uint8)
    for class_id, color in COLOR_MAP.items():
        color_mask[mask == class_id] = color
    return color_mask

def visualize(**images):
    """
    画像とマスクを横に並べて表示
    引数に image（CHW）、mask（HxW）、prediction（HxW）などを渡す
    """
    n = len(images)
    plt.figure(figsize=(6 * n, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(name.replace('_', ' ').title())

        # RGB画像（CHW形式）の場合
        if image.ndim == 3 and image.shape[0] == 3:
            plt.imshow(np.transpose(image, (1, 2, 0)))  # CHW → HWC

        # クラスIDマスク（HxW）の場合
        elif image.ndim == 2:
            plt.imshow(decode_mask(image))

        else:
            raise ValueError(f"Unsupported image shape: {image.shape}")
    plt.tight_layout()
    plt.show()



# def visualize(**images):
#     """Plot images in one row."""
#     n = len(images)
#     plt.figure(figsize=(16, 5))
#     for i, (name, image) in enumerate(images.items()):
#         plt.subplot(1, n, i + 1)
#         plt.xticks([])
#         plt.yticks([])
#         plt.title(" ".join(name.split("_")).title())

#         # If it's an image, plot it as RGB
#         if name == "image":
#             # Convert CHW to HWC for plotting
#             image = image.transpose(1, 2, 0)
#             plt.imshow(image)
#         else:
#             plt.imshow(image, cmap="tab20")
#     plt.show()

ValueError: cannot reshape array of size 1 into shape (1,1,3)

In [23]:
# ==========================
# データセットの初期化（データ拡張なし）
# ==========================
dataset = MedicalDataset(x_train_dir, y_train_dir)

# ==========================
# 最初のサンプルを取得
# ==========================
image_tensor, mask_tensor = dataset[0]

# ==========================
# 情報表示
# ==========================
print(f"Image shape: {image_tensor.shape}")  # (3, H, W)
print(f"Mask shape: {mask_tensor.shape}")    # (H, W)
print(f"Unique mask values: {torch.unique(mask_tensor)}")

# ==========================
# 可視化（テンソル → NumPy）
# ==========================
image = image_tensor.numpy().transpose(1, 2, 0)  # (H, W, 3)
mask = mask_tensor.numpy()                      # (H, W)

visualize(image=image, mask=mask)


# dataset = Dataset(x_train_dir, y_train_dir)
# image, mask = dataset[0]
# print(f"Mask shape: {mask.shape}")
# visualize(image=image, mask=mask)

ValueError: NumPy boolean array indexing assignment cannot assign 3 input values to the 650782 output values where the mask is true

# Augmentations
Data augmentation is a powerful technique to increase the amount of your data and prevent model overfitting.
If you not familiar with such trick read some of these articles:

 - [The Effectiveness of Data Augmentation in Image Classification using Deep Learning](http://cs231n.stanford.edu/reports/2017/pdfs/300.pdf)
 - [Data Augmentation | How to use Deep Learning when you have Limited Data](https://medium.com/nanonets/how-to-use-deep-learning-when-you-have-limited-data-part-2-data-augmentation-c26971dc8ced)
 - [Data Augmentation Experimentation](https://towardsdatascience.com/data-augmentation-experimentation-3e274504f04b)

Since our dataset is very small we will apply a large number of different augmentations:

 - horizontal flip
 - affine transforms
 - perspective transforms
 - brightness/contrast/colors manipulations
 - image bluring and sharpening
 - gaussian noise
 - random crops

All this transforms can be easily applied with [**Albumentations**](https://github.com/albu/albumentations/) - fast augmentation library. For detailed explanation of image transformations you can look at [**kaggle salt segmentation exmaple**](https://github.com/albu/albumentations/blob/master/notebooks/example_kaggle_salt.ipynb) provided by [**Albumentations**](https://github.com/albu/albumentations/) authors.

In [ ]:
# training set images augmentation
def get_training_augmentation():
    train_transform = [
        A.HorizontalFlip(p=0.5),
        A.ShiftScaleRotate(
            scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0
        ),
        A.PadIfNeeded(min_height=320, min_width=320, always_apply=True),
        A.RandomCrop(height=320, width=320, always_apply=True),
        A.GaussNoise(p=0.2),
        A.Perspective(p=0.5),
        A.OneOf(
            [
                A.CLAHE(p=1),
                A.RandomBrightnessContrast(p=1),
                A.RandomGamma(p=1),
            ],
            p=0.9,
        ),
        A.OneOf(
            [
                A.Sharpen(p=1),
                A.Blur(blur_limit=3, p=1),
                A.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),
        A.OneOf(
            [
                A.RandomBrightnessContrast(p=1),
                A.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
    ]
    return A.Compose(train_transform)


def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        A.PadIfNeeded(384, 480),
    ]
    return A.Compose(test_transform)

In [ ]:
# Visualize resulted augmented images and masks
augmented_dataset = Dataset(
    x_train_dir,
    y_train_dir,
    augmentation=get_training_augmentation(),
)

# Visualizing the same image with different random transforms
for i in range(3):
    image, mask = augmented_dataset[3]
    print(f"Mask shape: {mask.shape}")
    print(np.unique(mask))
    visualize(image=image, mask=mask)

In [ ]:
train_dataset = Dataset(
    x_train_dir,
    y_train_dir,
    augmentation=get_training_augmentation(),
)

valid_dataset = Dataset(
    x_valid_dir,
    y_valid_dir,
    augmentation=get_validation_augmentation(),
)

test_dataset = Dataset(
    x_test_dir,
    y_test_dir,
    augmentation=get_validation_augmentation(),
)

# Change to > 0 if not on Windows machine
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

# Create model and train

In [ ]:
# Some training hyperparameters
EPOCHS = 50
T_MAX = EPOCHS * len(train_loader)
# Always include the background as a class
OUT_CLASSES = len(train_dataset.CLASSES)

In [ ]:
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
import torch
from torch.optim import lr_scheduler


class CamVidModel(pl.LightningModule):
    def __init__(self, arch, encoder_name, in_channels, out_classes, **kwargs):
        super().__init__()
        self.model = smp.create_model(
            arch,
            encoder_name=encoder_name,
            in_channels=in_channels,
            classes=out_classes,
            **kwargs,
        )

        # Preprocessing parameters for image normalization
        params = smp.encoders.get_preprocessing_params(encoder_name)
        self.number_of_classes = out_classes
        self.register_buffer("std", torch.tensor(params["std"]).view(1, 3, 1, 1))
        self.register_buffer("mean", torch.tensor(params["mean"]).view(1, 3, 1, 1))

        # Loss function for multi-class segmentation
        self.loss_fn = smp.losses.DiceLoss(smp.losses.MULTICLASS_MODE, from_logits=True)

        # Step metrics tracking
        self.training_step_outputs = []
        self.validation_step_outputs = []
        self.test_step_outputs = []

    def forward(self, image):
        # Normalize image
        image = (image - self.mean) / self.std
        mask = self.model(image)
        return mask

    def shared_step(self, batch, stage):
        image, mask = batch

        # Ensure that image dimensions are correct
        assert image.ndim == 4  # [batch_size, channels, H, W]

        # Ensure the mask is a long (index) tensor
        mask = mask.long()

        # Mask shape
        assert mask.ndim == 3  # [batch_size, H, W]

        # Predict mask logits
        logits_mask = self.forward(image)

        assert (
            logits_mask.shape[1] == self.number_of_classes
        )  # [batch_size, number_of_classes, H, W]

        # Ensure the logits mask is contiguous
        logits_mask = logits_mask.contiguous()

        # Compute loss using multi-class Dice loss (pass original mask, not one-hot encoded)
        loss = self.loss_fn(logits_mask, mask)

        # Apply softmax to get probabilities for multi-class segmentation
        prob_mask = logits_mask.softmax(dim=1)

        # Convert probabilities to predicted class labels
        pred_mask = prob_mask.argmax(dim=1)

        # Compute true positives, false positives, false negatives, and true negatives
        tp, fp, fn, tn = smp.metrics.get_stats(
            pred_mask, mask, mode="multiclass", num_classes=self.number_of_classes
        )

        return {
            "loss": loss,
            "tp": tp,
            "fp": fp,
            "fn": fn,
            "tn": tn,
        }

    def shared_epoch_end(self, outputs, stage):
        # Aggregate step metrics
        tp = torch.cat([x["tp"] for x in outputs])
        fp = torch.cat([x["fp"] for x in outputs])
        fn = torch.cat([x["fn"] for x in outputs])
        tn = torch.cat([x["tn"] for x in outputs])

        # Per-image IoU and dataset IoU calculations
        per_image_iou = smp.metrics.iou_score(
            tp, fp, fn, tn, reduction="micro-imagewise"
        )
        dataset_iou = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")

        metrics = {
            f"{stage}_per_image_iou": per_image_iou,
            f"{stage}_dataset_iou": dataset_iou,
        }

        self.log_dict(metrics, prog_bar=True)

    def training_step(self, batch, batch_idx):
        train_loss_info = self.shared_step(batch, "train")
        self.training_step_outputs.append(train_loss_info)
        return train_loss_info

    def on_train_epoch_end(self):
        self.shared_epoch_end(self.training_step_outputs, "train")
        self.training_step_outputs.clear()

    def validation_step(self, batch, batch_idx):
        valid_loss_info = self.shared_step(batch, "valid")
        self.validation_step_outputs.append(valid_loss_info)
        return valid_loss_info

    def on_validation_epoch_end(self):
        self.shared_epoch_end(self.validation_step_outputs, "valid")
        self.validation_step_outputs.clear()

    def test_step(self, batch, batch_idx):
        test_loss_info = self.shared_step(batch, "test")
        self.test_step_outputs.append(test_loss_info)
        return test_loss_info

    def on_test_epoch_end(self):
        self.shared_epoch_end(self.test_step_outputs, "test")
        self.test_step_outputs.clear()

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=2e-4)
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=1e-5)
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "step",
                "frequency": 1,
            },
        }

In [ ]:
model = CamVidModel("FPN", "resnext50_32x4d", in_channels=3, out_classes=OUT_CLASSES)

# Training

In [ ]:
trainer = pl.Trainer(max_epochs=EPOCHS, log_every_n_steps=1)

trainer.fit(
    model,
    train_dataloaders=train_loader,
    val_dataloaders=valid_loader,
)

# Validation and test metrics

In [ ]:
# run validation dataset
valid_metrics = trainer.validate(model, dataloaders=valid_loader, verbose=False)
print(valid_metrics)

In [ ]:
# run test dataset
test_metrics = trainer.test(model, dataloaders=test_loader, verbose=False)
print(test_metrics)

# Result visualization

In [ ]:
import numpy as np

# Fetch a batch from the test loader
images, masks = next(iter(test_loader))

# Switch the model to evaluation mode
with torch.no_grad():
    model.eval()
    logits = model(images)  # Get raw logits from the model

# Apply softmax to get class probabilities
# Shape: [batch_size, num_classes, H, W]

pr_masks = logits.softmax(dim=1)
# Convert class probabilities to predicted class labels
pr_masks = pr_masks.argmax(dim=1)  # Shape: [batch_size, H, W]

# Visualize a few samples (image, ground truth mask, and predicted mask)
for idx, (image, gt_mask, pr_mask) in enumerate(zip(images, masks, pr_masks)):
    if idx <= 4:  # Visualize first 5 samples
        plt.figure(figsize=(12, 6))

        # Original Image
        plt.subplot(1, 3, 1)
        plt.imshow(
            image.cpu().numpy().transpose(1, 2, 0)
        )  # Convert CHW to HWC for plotting
        plt.title("Image")
        plt.axis("off")

        # Ground Truth Mask
        plt.subplot(1, 3, 2)
        plt.imshow(gt_mask.cpu().numpy(), cmap="tab20")  # Visualize ground truth mask
        plt.title("Ground truth")
        plt.axis("off")

        # Predicted Mask
        plt.subplot(1, 3, 3)
        plt.imshow(pr_mask.cpu().numpy(), cmap="tab20")  # Visualize predicted mask
        plt.title("Prediction")
        plt.axis("off")

        # Show the figure
        plt.show()
    else:
        break